In [1]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 15.6 MB/s 
     |████████████████████████████████| 7.6 MB 58.2 MB/s 
     |████████████████████████████████| 182 kB 76.3 MB/s 


In [4]:
# from transformers import RobertaTokenizer,BertTokenizer, AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
# from transformers import (
#     AdamW,
#     get_linear_schedule_with_warmup
# )
# """
# # Load the tokenizer.
# print('Loading Roberta tokenizer...')
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)"""

# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# config = AutoConfig.from_pretrained('bert-base-uncased',
#                                             num_labels=101,
#                                             hidden_dropout_prob=0.15)



In [5]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import RobertaTokenizer,BertTokenizer, AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
from transformers import (
    AdamW,
    get_linear_schedule_with_warmup
)
output_dir = '/content/drive/MyDrive/Fall_2022/NLP/Project2/Codes/fewshot_data/model_save_101_epoch/'

tokenizer = AutoTokenizer.from_pretrained(output_dir+'tokenizer/')
model = AutoModelForSequenceClassification.from_pretrained(output_dir)


# Copy the model to the GPU.
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.15, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [6]:
import argparse
import torch
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
import pandas as pd
import numpy as np

In [7]:
dataset=pd.read_csv("/content/drive/MyDrive/Fall_2022/NLP/Project2/Codes/fewshot_data/train.csv")

In [8]:
reviews = dataset.loc[:,['real_label','review']]

In [9]:
reviews

,real_label,review
0,44,I have always been a fan of the Lewis and Clar...
1,44,I started reading this book during a three-wee...
2,44,This cable is well-made and feels sturdier tha...
3,44,"This kibble is expensive, but the cats like it..."
4,44,This device is exactly what we needed for our ...
...,...,...
49995,46,Dumbo is one of disney's most beloved and clas...
49996,46,I haven't played anygood sport games yet and i...
49997,46,"This game is loads of fun, i really liked this..."
49998,46,Gabriel is one of the funniest comedians i've ...


In [10]:
labels = reviews.real_label.values

In [11]:
len(labels)

50000

In [12]:
sentences = reviews.review.values

In [13]:
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

Token indices sequence length is longer than the specified maximum sequence length for this model (644 > 512). Running this sequence through the model will result in indexing errors


 Original:  I have always been a fan of the Lewis and Clark (L&C) expedition and have read numerous books on that era.  I even road tripped along the Missouri River to the river's source in Montana (bypassing North Dakota and eastern Montana due to repeated tornadoes that summer), stopping along all the historical signs.  I'm a fan because I understand the courage it took to explore unknown lands with potential violent inhabitants, but I also understand the significance of American expansion to the Pacific via the Northwest passage.  Opening up these lands is one of Jefferson's biggest legacies as president.

What Julie Fenster does here is not just summarize the L&C expedition, though.  She describes the young America at the turn of the 19th century.  Pioneers were moving westward, but Spain controlled the western lands. She gives short biographies of the players, the governors, kings and explorers of the era.  It was a time of great hostilities.  Fenster portrays Jefferson as a man f

In [14]:
input_ids = []
attention_masks = []

# For every sentence...
for s in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        s,                      # Sentence to encode.
                        truncation=True,
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        padding='max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        return_overflowing_tokens=False,
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

In [15]:
input_ids[0]

tensor([  101,  1045,  2031,  2467,  2042,  1037,  5470,  1997,  1996,  4572,
         1998,  5215,  1006,  1048,  1004,  1039,  1007,  5590,  1998,  2031,
         3191,  3365,  2808,  2006,  2008,  3690,  1012,  1045,  2130,  2346,
        21129,  2247,  1996,  5284,  2314,  2000,  1996,  2314,  1005,  1055,
         3120,  1999,  8124,  1006, 11826,  2075,  2167,  7734,  1998,  2789,
         8124,  2349,  2000,  5567, 22668,  2008,  2621,  1007,  1010,  7458,
         2247,  2035,  1996,  3439,  5751,  1012,  1045,  1005,  1049,  1037,
         5470,  2138,  1045,  3305,  1996,  8424,  2009,  2165,  2000,  8849,
         4242,  4915,  2007,  4022,  6355,  4864,  1010,  2021,  1045,  2036,
         3305,  1996,  7784,  1997,  2137,  4935,  2000,  1996,  3534,  3081,
         1996,  4514,  6019,  1012,  3098,  2039,  2122,  4915,  2003,  2028,
         1997,  7625,  1005,  1055,  5221,  4190, 20499,  2004,  2343,  1012,
         2054,  7628, 21713,  6238,  2515,  2182,  2003,  2025, 

In [16]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

45,000 training samples
5,000 validation samples


In [17]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size, # Trains with this batch size.
            num_workers=2
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size, # Evaluate with this batch size.
            num_workers=2
        )

In [18]:
# from transformers import RobertaForSequenceClassification,BertForSequenceClassification, AdamW, BertConfig

# # Load BertForSequenceClassification, the pretrained BERT model with a single 
# # linear classification layer on top. 

# model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

# """
# model = RobertaForSequenceClassification.from_pretrained(
#     "roberta-base", # Use the 12-layer BERT model, with an uncased vocab.
#     num_labels = 100, # The number of output labels--2 for binary classification.
#              # You can increase this for multi-class tasks.   
#     output_attentions = False, # Whether the model returns attentions weights.
#     output_hidden_states = True, # Whether the model returns all hidden-states.
# )"""

# # Tell pytorch to run this model on the GPU.
# model.to(device)

In [19]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [20]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 1

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
#total_steps = len(train_dataloader) * epochs
gradient_accumulation_steps=1
n_gpu=1
#total_steps= ((len(train_dataloader) // (batch_size * max(1, n_gpu)))// gradient_accumulation_steps* float(epochs))
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [21]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [22]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [23]:
# logits_known = []
# logits_unknown= []
# import random
# import numpy as np
# import time
# import torch.nn as nn 
# # This training code is based on the `run_glue.py` script here:
# # https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# # Set the seed value all over the place to make this reproducible.
# seed_val = 44

# random.seed(seed_val)
# np.random.seed(seed_val)
# torch.manual_seed(seed_val)
# torch.cuda.manual_seed_all(seed_val)

# # We'll store a number of quantities such as training and validation loss, 
# # validation accuracy, and timings.
# training_stats = []

# # Measure the total training time for the whole run.
# total_t0 = time.time()

# # For each epoch...
# cls = list(range(0,100))
# for epoch_i in range(0, epochs):
    
#     # ========================================
#     #               Training
#     # ========================================
    
#     # Perform one full pass over the training set.

#     print("")
#     print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
#     print('Training...')
#     unknown_cls = random.sample(cls, 10)

#     # Measure how long the training epoch takes.
#     t0 = time.time()

#     # Reset the total loss for this epoch.
#     total_train_loss = 0

#     # Put the model into training mode. Don't be mislead--the call to 
#     # `train` just changes the *mode*, it doesn't *perform* the training.
#     # `dropout` and `batchnorm` layers behave differently during training
#     # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
#     model.train()

#     # For each batch of training data...
#     for step, batch in enumerate(train_dataloader):

#         # Progress update every 40 batches.
#         if step % 40 == 0 and not step == 0:
#             # Calculate elapsed time in minutes.
#             elapsed = format_time(time.time() - t0)
            
#             # Report progress.
#             print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

#         # Unpack this training batch from our dataloader. 
#         #
#         # As we unpack the batch, we'll also copy each tensor to the GPU using the 
#         # `to` method.
#         #
#         # `batch` contains three pytorch tensors:
#         #   [0]: input ids 
#         #   [1]: attention masks
#         #   [2]: labels 
#         b_input_ids = batch[0].to(device)
#         b_input_mask = batch[1].to(device)
#         b_labels = batch[2].to(device)
#         # print(b_labels,b_input_ids)
#         # print(b_labels)
#         unknown_input_ids = []
#         unknown_input_mask = []
#         unknown_labels = []

#         known_input_ids = []
#         known_input_mask = []
#         known_labels = []

#         for i in range(len(batch[2])):
#           if batch[2][i] in unknown_cls:
#             re = torch.randperm(100)
#             temp = batch[0][i][re]
#             unknown_input_ids.append(temp.numpy())
#             temp = batch[1][i][re]
#             unknown_input_mask.append(temp.numpy())
#             unknown_labels.append(100)
#           else:
#             known_input_ids.append(batch[0][i].numpy())
#             known_input_mask.append(batch[1][i].numpy())
#             known_labels.append(batch[2][i])
#         # print(unknown_input_ids)

#         unknown_input_ids = torch.tensor(unknown_input_ids).to(device)
#         unknown_input_mask = torch.tensor(unknown_input_mask).to(device)
#         unknown_labels = torch.tensor(unknown_labels).to(device)

#         known_input_ids = torch.tensor(known_input_ids).to(device)
#         known_input_mask = torch.tensor(known_input_mask).to(device)
#         known_labels = torch.tensor(known_labels).to(device)

#         # print(unknown_labels,unknown_input_mask,unknown_input_ids)

#         # print(len(known_input_ids),len(known_input_mask), len(known_labels))


#         # Always clear any previously calculated gradients before performing a
#         # backward pass. PyTorch doesn't do this automatically because 
#         # accumulating the gradients is "convenient while training RNNs". 
#         # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
#         model.zero_grad()        

#         # Perform a forward pass (evaluate the model on this training batch).
#         # In PyTorch, calling `model` will in turn call the model's `forward` 
#         # function and pass down the arguments. The `forward` function is 
#         # documented here: 
#         # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
#         # The results are returned in a results object, documented here:
#         # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
#         # Specifically, we'll get the loss (because we provided labels) and the
#         # "logits"--the model outputs prior to activation.
#         # result = model(b_input_ids, 
#         #                token_type_ids=None, 
#         #                attention_mask=b_input_mask, 
#         #                labels=b_labels,
#         #                return_dict=True)
        
#         result_known = model(known_input_ids, 
#                        token_type_ids=None, 
#                        attention_mask=known_input_mask, 
#                        labels=known_labels,
#                        return_dict=True)
#         logits_known = result_known.logits
#         logits_unknown= []
#         if len(unknown_labels) >0 :
#           result_unknown = model(unknown_input_ids, 
#                        token_type_ids=None, 
#                        attention_mask=unknown_input_mask, 
#                        labels=unknown_labels,
#                        return_dict=True)
#           loss_unknown = result_unknown.loss
#           logits_unknown = result_unknown.logits
#         break
#     break


In [24]:
import random
import numpy as np
import time
import torch.nn as nn 
import torch.nn.functional as F
import math
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 44

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...

for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        cls = list(range(0,100))
        unknown_cls = random.sample(cls, 2)
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # print(b_labels,b_input_ids)
        # print(b_labels)
        unknown_input_ids = []
        unknown_input_mask = []
        unknown_labels = []

        known_input_ids = []
        known_input_mask = []
        known_labels = []

        for i in range(len(batch[2])):
          if batch[2][i] in unknown_cls:
            # re = torch.randperm(100)
            # temp = batch[0][i][re]
            unknown_input_ids.append(batch[0][i].numpy())
            # temp = batch[1][i][re]
            unknown_input_mask.append(batch[1][i].numpy())
            unknown_labels.append(100)
          else:
            known_input_ids.append(batch[0][i].numpy())
            known_input_mask.append(batch[1][i].numpy())
            known_labels.append(batch[2][i])
        # print(unknown_input_ids)

        unknown_input_ids = torch.tensor(unknown_input_ids).to(device)
        unknown_input_mask = torch.tensor(unknown_input_mask).to(device)
        unknown_labels = torch.tensor(unknown_labels).to(device)

        known_input_ids = torch.tensor(known_input_ids).to(device)
        known_input_mask = torch.tensor(known_input_mask).to(device)
        known_labels = torch.tensor(known_labels).to(device)

        # print(unknown_labels,unknown_input_mask,unknown_input_ids)

        # print(len(known_input_ids),len(known_input_mask), len(known_labels))


        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. The `forward` function is 
        # documented here: 
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation.
        # result = model(b_input_ids, 
        #                token_type_ids=None, 
        #                attention_mask=b_input_mask, 
        #                labels=b_labels,
        #                return_dict=True)
        
        result_known = model(known_input_ids, 
                       token_type_ids=None, 
                       attention_mask=known_input_mask, 
                       labels=known_labels,
                       return_dict=True)
        logits_unknown= []
        if len(unknown_labels) >0 :
          result_unknown = model(unknown_input_ids, 
                       token_type_ids=None, 
                       attention_mask=unknown_input_mask, 
                       labels=unknown_labels,
                       return_dict=True)
          loss_unknown = result_unknown.loss
          logits_unknown = result_unknown.logits
          softmax_unknown =  F.softmax(logits_unknown)
          unknow_loss = 0
          for index,i in enumerate(softmax_unknown):
            for cls,j in enumerate(i):
              if cls not in unknown_cls:
                unknow_loss += j* math.log(j)


        loss_known = result_known.loss
        

        logits_known = result_known.logits
        
        eps = 1e-7

        loss_crossentrophy_function = nn.CrossEntropyLoss()
        loss_known = loss_crossentrophy_function(logits_known,known_labels)



      #   if len(logits_unknown) >0 and len(logits_known) > 0:

      #     kthlogits1 = logits_unknown[: , len(logits_known[0])-1:len(logits_known[0])].clone()
      # #  print("kthlogits before Relu   ",kthlogits1)
      #     ReLU = nn.ReLU()
      #     kthlogits1 = ReLU(kthlogits1)
      # #  print("kthlogits after Relu   ",kthlogits1)
      #     kthloss1 = -1 * torch.log(kthlogits1 + eps)
      #  # print("kthloss  ",kthloss1)
      #     kthloss1 = torch.sum(kthloss1)
      #     kthloss1 = torch.sum(kthloss1)/len(known_labels)
      #  # print("kthloss  ",kthloss1)
      #   #print("logits : ",logits)
      #   #print(len(logits))
      #     kthlogits2 = ReLU(kthlogits2)
      #     kthloss2 = -1 * torch.log(kthlogits2 + eps)
      #     kthloss2 = torch.sum(kthloss2)/len(unknown_labels)
      #     loss = loss_known + (0.4 * kthloss1) + (0.999 * kthloss2)
      #   else:
        loss = loss_known
        lambda_var  = 3
        if len(unknown_labels) >0:
          loss += lambda_var*unknow_loss

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()
    


    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = result.loss
        logits = result.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 1 ========
Training...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:104: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:148: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  Batch    40  of  2,813.    Elapsed: 0:00:58.
  Batch    80  of  2,813.    Elapsed: 0:01:56.
  Batch   120  of  2,813.    Elapsed: 0:02:55.
  Batch   160  of  2,813.    Elapsed: 0:03:55.
  Batch   200  of  2,813.    Elapsed: 0:04:56.
  Batch   240  of  2,813.    Elapsed: 0:05:58.
  Batch   280  of  2,813.    Elapsed: 0:06:59.
  Batch   320  of  2,813.    Elapsed: 0:08:01.
  Batch   360  of  2,813.    Elapsed: 0:09:02.
  Batch   400  of  2,813.    Elapsed: 0:10:04.
  Batch   440  of  2,813.    Elapsed: 0:11:06.
  Batch   480  of  2,813.    Elapsed: 0:12:07.
  Batch   520  of  2,813.    Elapsed: 0:13:08.
  Batch   560  of  2,813.    Elapsed: 0:14:10.
  Batch   600  of  2,813.    Elapsed: 0:15:11.
  Batch   640  of  2,813.    Elapsed: 0:16:13.
  Batch   680  of  2,813.    Elapsed: 0:17:14.
  Batch   720  of  2,813.    Elapsed: 0:18:16.
  Batch   760  of  2,813.    Elapsed: 0:19:17.
  Batch   800  of  2,813.    Elapsed: 0:20:19.
  Batch   840  of  2,813.    Elapsed: 0:21:20.
  Batch   880

In [25]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = '/content/drive/MyDrive/Fall_2022/NLP/Project2/Codes/fewshot_data/model_save_openset_2/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
if not os.path.exists(output_dir+'tokenizer/'):
    os.makedirs(output_dir+'tokenizer/')

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir+'tokenizer/')

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))


Saving model to /content/drive/MyDrive/Fall_2022/NLP/Project2/Codes/fewshot_data/model_save_openset_2/


('/content/drive/MyDrive/Fall_2022/NLP/Project2/Codes/fewshot_data/model_save_openset_2/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/Fall_2022/NLP/Project2/Codes/fewshot_data/model_save_openset_2/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/Fall_2022/NLP/Project2/Codes/fewshot_data/model_save_openset_2/tokenizer/vocab.txt',
 '/content/drive/MyDrive/Fall_2022/NLP/Project2/Codes/fewshot_data/model_save_openset_2/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/Fall_2022/NLP/Project2/Codes/fewshot_data/model_save_openset_2/tokenizer/tokenizer.json')

In [26]:
# !ls /content/drive/MyDrive/Fall_2022/NLP/Project2/Codes/fewshot_data/model_save_openset/

In [27]:
output_dir = '/content/drive/MyDrive/Fall_2022/NLP/Project2/Codes/fewshot_data/model_save_openset_2/'

tokenizer = AutoTokenizer.from_pretrained(output_dir+'tokenizer/')
model = AutoModelForSequenceClassification.from_pretrained(output_dir)


# Copy the model to the GPU.
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.15, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [28]:
import pandas as pd
import regex as re
import numpy as np
df = pd.read_csv("/content/drive/MyDrive/Fall_2022/NLP/Project2/Codes/fewshot_data/ood_test_data_small.csv")
df = df[['real_label','review']]

In [29]:
arr_real_labels = [i for i in range(100,10000)]
df['real_label'] = df['real_label'].replace(arr_real_labels,100)
df=df.loc[df.real_label.values<=100]

In [30]:

# Create sentence and label lists
#
labels = df.real_label.values
sentence1 = df.review.values
#sentence1=sentences
#labels = dataset_filtered.real_label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent1 in zip(sentence1):
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    #print(sent1)
    encoded_dict = tokenizer.encode_plus(
                        sent1[0],# Sentence to encode.
                        truncation=True,
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        padding='max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        return_overflowing_tokens=False,
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)



# Set the batch size.  
batch_size = 16

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [31]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions.
      result = model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask,
                     return_dict=True)

  logits = result.logits

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  logits[:,100] = logits[:,100] + 8.5
  pred_labels = np.argmax(logits, axis=1)
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.extend(pred_labels.tolist())
  true_labels.extend(label_ids.tolist())

print('    DONE.')

Predicting labels for 20,000 test sentences...
    DONE.


In [32]:
from sklearn.metrics import classification_report

result_report= classification_report(true_labels, predictions, digits=5)
print(result_report)

              precision    recall  f1-score   support

           0    0.45745   0.86000   0.59722       100
           1    0.67626   0.94000   0.78661       100
           2    0.32593   0.88000   0.47568       100
           3    0.74809   0.98000   0.84848       100
           4    0.18576   0.60000   0.28369       100
           5    0.42614   0.75000   0.54348       100
           6    0.39759   0.66000   0.49624       100
           7    0.83051   0.98000   0.89908       100
           8    0.47980   0.95000   0.63758       100
           9    0.44949   0.89000   0.59732       100
          10    0.61146   0.96000   0.74708       100
          11    0.46629   0.83000   0.59712       100
          12    0.38554   0.96000   0.55014       100
          13    0.86239   0.94000   0.89952       100
          14    0.56731   0.59000   0.57843       100
          15    0.71533   0.98000   0.82700       100
          16    0.77193   0.88000   0.82243       100
          17    0.73109   0

In [33]:
df2 = pd.read_csv("/content/drive/MyDrive/Fall_2022/NLP/Project2/Codes/fewshot_data/ood_test_data_small.csv")
df2["N_plus_one_prediction"] = predictions
df2['N_plus_one_prediction'] = df2['N_plus_one_prediction'].replace([100], 'OOD')
combined_df = df2
number_of_known_labels = 100

In [34]:
def compute_precision_recall_for_known_classes(given_label):
    TP = combined_df[(combined_df["real_label"] == given_label) & (combined_df["N_plus_one_prediction"] == given_label)].shape[0]
    FP = combined_df[(combined_df["real_label"] != given_label) & (combined_df["N_plus_one_prediction"] == given_label)].shape[0]
    FN = combined_df[(combined_df["real_label"] == given_label) & (combined_df["N_plus_one_prediction"] != given_label)].shape[0]
    if(TP == 0):
        P = 0
        R = 0
    else:
        P = round(100*TP/(TP + FP), 2)
        R = round(100*TP/(TP + FN),2)
    return P, R

def compute_precision_recall_for_OOD():
    TP = combined_df[(combined_df["real_label"] >= number_of_known_labels) & (combined_df["N_plus_one_prediction"] == "OOD")].shape[0]
    FP = combined_df[(combined_df["real_label"] < number_of_known_labels) & (combined_df["N_plus_one_prediction"] == "OOD")].shape[0]
    FN = combined_df[(combined_df["real_label"] >= number_of_known_labels) & (combined_df["N_plus_one_prediction"] != "OOD")].shape[0]
    if(TP == 0):
        P = 0
        R = 0
    else:
        P = 100*TP/(TP + FP)
        R = 100*TP/(TP + FN)
    return P, R

def compute_performance_metrics():
    from statistics import mean
    
    precisions = []
    recalls = []
    for known_label in range(number_of_known_labels):
        p, r = compute_precision_recall_for_known_classes(known_label)
        precisions.append(p)
        recalls.append(r)
    
    precision_known, recall_known = round(mean(precisions),2), round(mean(recalls),2)
    f1_known = round(2*precision_known*recall_known/(precision_known + recall_known),2)
    
    precision_ood, recall_ood = compute_precision_recall_for_OOD()
    f1_ood = round(2*precision_ood*recall_ood/(precision_ood + recall_ood),2)
    
    precisions.append(precision_ood)
    recalls.append(recall_ood)
    
    precision, recall = round(mean(precisions),2), round(mean(recalls),2)
    f1 = round(2*precision*recall/(precision + recall),2)
    
    return {
        "known": {
            "precision": precision_known,
            "recall": recall_known,
            "f1": f1_known,
        },
        "ood": {
            "precision": round(precision_ood,2),
            "recall": round(recall_ood,2),
            "f1": f1_ood,
        },
        "overall": {
            "precision": precision,
            "recall": recall,
            "f1": f1,
        }
        
    }

def print_performance_metrics():
    performance_dict = compute_performance_metrics()
    print("Precision  Recall f1")
    print("Known:",performance_dict["known"]["precision"],performance_dict["known"]["recall"],performance_dict["known"]["f1"], end = " \n")
    print("odd:",performance_dict["ood"]["precision"],performance_dict["ood"]["recall"],performance_dict["ood"]["f1"], end = " \n")
    print("overall:",performance_dict["overall"]["precision"],performance_dict["overall"]["recall"],performance_dict["overall"]["f1"], end = " \n")

In [35]:
print_performance_metrics()

Precision  Recall f1
Known: 55.48 84.79 67.07 
odd: 89.85 24.0 37.88 
overall: 55.82 84.19 67.13 
